**LING 484: Final Project**

Duygu Demiray & Elifnur Ulusoy (Group 5)

Raincheck: a weather app which tells you what to wear/pack

Important note: this does NOT work in Google Colab because we have to take input from a mic. Please run it on your preferred environment. We used Jupyter Notebooks. 

Step 1: Listen to user commands (I know voice commands are not 
necessary, but I still wanted to give it a try)
[source](https://sweetcode.io/how-build-digital-virtual-assistant-python/)

In [5]:
import speech_recognition as sr 
import pyaudio
from time import ctime
import time
import os
! pip install gtts
from gtts import gTTS
import requests, json

In [6]:
def listen():
    """Takes voice input from the user"""
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("I am listening...")
        audio = r.listen(source)
    data = ""
    try:
        data = r.recognize_google(audio)
        print("You said: " + data)
    except sr.UnknownValueError:
        print("I don't understand. Could you repeat that?")
    except sr.RequestError as e:
        print("Sorry, I don't understand.; {0}".format(e))
    return data

In [25]:
!pip install flair
from flair.data import Sentence
from flair.models import SequenceTagger

def Tagger(data):
    """Performs entity recognition using the package Flair"""
    sentence = Sentence(data)
    tagger = SequenceTagger.load('ner-ontonotes-fast')
    tagger.predict(sentence)
    return sentence.get_spans()

A problem with the function below is that it will definitely overgenerate. The sequence tagger used inside Tagger() can tag 18 classes, whereas we are only interested in the following classes:

* DATE: date value
* GPE: geo-political entity 
* LOC: location name
* TIME: time value

Actually, we only need TIME at this point. We can try to get location-based weather input and then we can use GPE & LOC too. We will need to pull some tricks to use DATE too but I don't really think that is necessary. 

We can modify Tagger() such that it only returns sentences w/ time & location information and throws some sort of error message otherwise. 

In [1]:
def NoName():
    """Takes voice input and returns time & location information"""
    time.sleep(2)
    listening = True
    while listening == True:
        data = listen()
        listening = Tagger(data)
    return listening

NoName()

NameError: name 'time' is not defined

In [18]:
def GenerateOutput():
    parse = NoName()
     
#if we write this properly we are finished. run all cells above this and nothing below for now.     

In [8]:

def digital_assistant(data):
  
    if "how are you" in data:
        listening = True
        respond("Very well, thank you!")

    if "what time is it" in data:
        listening = True
        respond(ctime())

    if "who are you" in data:
        listening = True
        respond("I am Raincheck, at your service.")
        
    if "stop listening" in data:
        listening = False
        print('Listening stopped')
        return listening
    #return listening

If we could put the digital_assistant inside a loop until we hear 'stop listening' that would be great. I am going to try that with a new cell below because I don't want to accidently ruin everything.
What we need to do, more importantly, is to use the dictionary we obtained in Step 2 inside the digital_assistant. As soon as we get an input asking about weather, we'll need to refer to the dictionary to check out the weather and respond accordingly. 

{'cur_weather': 70,
 'morning': 74,
 'afternoon': 80,
 'evening': 71,
 'overnight': 63,
 'tomorrow': 80,
 'tomorrow_night': 62,
 'day_after': 79,
 'day_after_n': 63,
 'after_after': 70,
 'after_after_n': 59,
 'idk': 68,
 'idk_n': 60}

In [ ]:
#this is pseudocode. 

if "what should i wear today" in data:
    listening = True
    if in dictionary(cur_weather<25): #i don't know the syntax for this
        respond('go grab a jacket') 
    elif...
if "what's the weather like this afternoon" in data:
    listening = True
    if in dictionary(afternoon>20):
        respond ('wear a tshirt')
    elif...


In [5]:
#this confirms whether your system detects the mic. it should NOT return an empty list. 

sr.Microphone.list_microphone_names() 

['Microsoft Sound Mapper - Input',
 'Microphone (Realtek High Defini',
 'Microsoft Sound Mapper - Output',
 'Speakers (Realtek High Definiti',
 'Primary Sound Capture Driver',
 'Microphone (Realtek High Definition Audio)',
 'Primary Sound Driver',
 'Speakers (Realtek High Definition Audio)',
 'Speakers (Realtek High Definition Audio)',
 'Microphone (Realtek High Definition Audio)',
 'Stereo Mix (Realtek HD Audio Stereo input)',
 'Microphone (Realtek HD Audio Mic input)',
 'Speakers (Realtek HD Audio output)']

In [10]:
time.sleep(2)
respond("Hello, what can I do for you?")
listening = True
while listening == True:
    data = listen()
    listening = digital_assistant(data)

Hello, what can I do for you?
I am listening...
You said: who are you
I am Raincheck, at your service.


some questions:
*   do we need multiple locations? should we let the user choose their location? (if so, find out how)

This will be difficult because the URL we are reading specifically fetches the weather for istanbul. I do not know how to go into the website AND change the location based on the users location. I think it might be a bit too much work, and we might need to read the website differently. We can get around to it if we have time. - D.

*   can we choose between F/C?

Step 2: Read into the Weather Channel website and get Today's Forecast and Daily Forecast. 

In [25]:
#elif's version of duygu's code -- to find out whether we can switch btwn F/C
#the code below only prints in C but doesn't really matter since it only shows weather for Istanbul

import re 
import urllib.request 

class AppURLopener(urllib.request.FancyURLopener): 
    version = "Mozilla/5.0"

opener = AppURLopener()
response = opener.open('https://weather.com/tr-TR/weather/today/l/33d1e415eb66f3e1ab35c3add45fccf4512715d329edbd91c806a6957e123b49')
file_1 = response.read().decode('utf-8') 

pattern_1 = re.compile("([0-9]+°)") 

matches_1 = re.findall(pattern_1, file_1) 

print(matches) #returns in celsius



<ipython-input-25-62a120dc3f7a>:10: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  opener = AppURLopener()


['22°', '17°', '23°', '26°', '22°', '18°', '22°', '17°', '15°', '22°', '22°', '21°', '20°', '19°', '17°', '27°', '17°', '26°', '17°', '21°', '15°', '20°', '16°']


In [ ]:
#code for switching between C/F
# celsius * 1.8 = fahrenheit - 32

#C to F
celsius = float(input("Enter temperature in celsius: "))
fahrenheit = (celsius * 9/5) + 32
print('%.2f Celsius is: %0.2f Fahrenheit' %(celsius, fahrenheit))

#F to C
fahrenheit = float(input("Enter temperature in fahrenheit: "))
celsius = (fahrenheit - 32) * 5/9
print('%.2f Fahrenheit is: %0.2f Celsius' %(fahrenheit, celsius))

In [89]:
import re 
import urllib.request 

class AppURLopener(urllib.request.FancyURLopener): 
    version = "Mozilla/5.0"

opener = AppURLopener()
response = opener.open('https://weather.com/weather/today/l/33d1e415eb66f3e1ab35c3add45fccf4512715d329edbd91c806a6957e123b49')
file_2 = response.read().decode('utf-8') 

pattern_2 = re.compile("([0-9]+)°") 

matches_2 = re.findall(pattern_2, file_2) 

print(matches_2) #returns in F.

 #this needs to be a function and you need to return the matches??



<ipython-input-89-5258cac921ad>:7: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  opener = AppURLopener()


['70', '62', '74', '80', '71', '63', '70', '62', '59', '70', '69', '68', '67', '65', '62', '80', '62', '79', '63', '70', '59', '68', '60']


matches is a list of numbers, and maybe indexing them will be easier than writing separate RegEx's. we can create a dictionary and then use that dictionary in the 3rd step.

this is a very dumb way and we're not doing much work w/ regex but it works, so i think we'll be fine. 

beware: I have tweaked with CleanList because of the website. if it does not work for you in the dictionary function below make while a<2 and while b<=10. 

In [90]:
def CleanList(matches):
  """Returns the relevant numbers from the scraped data""" 

  a=0
  while a<1:
    matches.pop(1)
    a+=1

  b=0
  while b<=8:
    matches.pop(5)
    b+=1 
  
  return matches

CleanList(matches_2)

['70', '74', '80', '71', '63', '80', '62', '79', '63', '70', '59', '68', '60']

In [93]:
#we are turning them to int's. for some reason this did not work when I put it inside CleanList

integer_map = map(int, matches_2) 
matches_int = list(integer_map)

matches_int

[70, 74, 80, 71, 63, 80, 62, 79, 63, 70, 59, 68, 60]

open the weather channel and look side-by-side.

[0-4] -> 0. current weather / 1. morning / 2. afternoon / 3. evening / 4. overnight 

[5-12] -> skip down to daily forecast in the website, which gives us the weather for the next four days. I kept the values for both day & night, but of course we can purge it further. 

now we need to match these with the list indeces. 

In [94]:
#maybe we should get rid of everything that comes after day_after_n. 

labels = ["cur_weather", "morning", "afternoon", "evening", "overnight", "tomorrow", "tomorrow_night", "day_after", "day_after_n", "after_after", "after_after_n", "idk", "idk_n"]

def CreateDictionary(matches, labels):
  """Creating a dictionary from the numbers obtained from the weather channel and times of the day"""

  weather = {}

  for k in range(len(labels)):
    weather[labels[k]] = matches[k]
  
  return weather

dictionary = CreateDictionary(matches_int, labels) #this may not work from time to time.  

Some problems with this: we are taking everything but we will possibly only use a limited part of it when we get input from the user, as they will ask what to wear for a specific time of the day/week. It would be better to have separate RegEx's that read the relevant parts of the website based on user input, but I couldn't think of/find a way to do that. 

In [95]:
dictionary

{'cur_weather': 70,
 'morning': 74,
 'afternoon': 80,
 'evening': 71,
 'overnight': 63,
 'tomorrow': 80,
 'tomorrow_night': 62,
 'day_after': 79,
 'day_after_n': 63,
 'after_after': 70,
 'after_after_n': 59,
 'idk': 68,
 'idk_n': 60}